In [6]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

import os
import glob

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width' , 1000)

In [7]:
# Variables
season = "2021-22"

In [8]:
all_players = pd.read_csv (r'./data/' + season + '/Raw_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Raw_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Raw_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Raw_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Raw_data/game_phases.csv')

# understat_player = pd.read_csv(r'./data/' + season + '/Raw_data/understat/players/understat_player.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Raw_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Raw_data/player_gameweek_history.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Raw_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])

In [9]:
# import os
# import glob
# import pandas as pd
# os.chdir(r'C:/Users/danie/OneDrive/Documents/GitHub/FPLAnlysis/data/2020-21/Raw_data/understat/teams/')

# extension = 'csv'
# all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# #combine all files in the list
# combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# #export to csv
# combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')
# combined_csv.head()

## Rename columns

In [10]:
player_gameweek_history.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,1,3,1,False,2021-08-13T19:00:00Z,2.0,0.0,1,90,0,0,0,2,0,0,0,0,0,1,0,11,11.8,0.0,0.0,1.2,50,0,153122,0,0
1,1,18,6,2,True,2021-08-22T15:30:00Z,0.0,2.0,2,90,0,0,0,2,0,0,0,0,0,3,0,17,24.2,0.0,0.0,2.4,50,-25816,144885,2627,28443
2,1,24,12,1,False,2021-08-28T11:30:00Z,5.0,0.0,3,90,0,0,0,5,0,0,0,0,0,5,0,20,43.0,0.0,0.0,4.3,49,-14559,136757,5164,19723
3,1,31,15,0,True,2021-09-11T14:00:00Z,1.0,0.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,49,6564,153130,35360,28796
4,1,43,5,0,False,2021-09-18T14:00:00Z,0.0,1.0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,49,-23847,129851,4822,28669


In [11]:
player_gameweek_history.rename(
    columns = {
        "element" : "player_id" , 
        "opponent_team" : "opponent_team_id" , 
        "fixture" : "fixture_id" 
        }, 
        inplace = True)

all_players.rename(
    columns = {
        "code" : "player_code", 
        "element_type" : "player_type_id" , 
        "minutes" : "total_minutes" , "id" : 
        "player_id" , "team" : "team_id" , 
        "points_per_game" : "PPG"
        }, 
        inplace=True)

all_teams.rename(
    columns = {
        "id" : "team_id" , 
        "code" : "team_code" , 
        "name" : "team_name" , 
        "short_name" : "team_short_name" , 
        "strength" : "team_strength" , 
        "form" : "team_form"
        }, 
        inplace = True)

all_events.rename(
    columns = {
        "id" : "gameweek_id"
        }, 
        inplace = True)

player_types.rename(
    columns = {
        "id" : "player_type_id" , 
        "singular_name" : "position_name" , 
        "singular_name_short" : "position_name_short"
        }, 
        inplace = True)

game_phases.rename(
    columns = {
        "id" : "phase_id"
        }, 
        inplace = True)

player_season_history.rename(
    columns = {
        "element_code" : "player_code"
        }, 
        inplace = True)

player_future_fixture.rename(
    columns = {
        "id" : "fixture_id" , 
        "team_h" : "home_team_id" , 
        "team_a" : "away_team_id" , 
        "events" : "gameweek_id" , 
        "code" : "fixture_code"
        }, 
        inplace = True)
        
#player_gameweek_history.rename(columns = {} , inplace = True)


## Drop columns

In [12]:
all_events.drop(
    columns = [
        "deadline_time_epoch"
        ], 
        inplace = True)

player_types.drop(
    columns = [
        "plural_name" , 
        "plural_name_short"
        ], 
        inplace = True)
        
all_players.drop(
    columns = [
        "squad_number" , 
        "special" 
        #"transfers_in" , 
        #"transfers_out" , 
        #"first_name" , 
        #"second_name"
        ], 
        inplace = True)

## Change date columns

In [13]:
all_players['news_added'] = all_players['news_added'].dt.tz_localize(None)
all_events['deadline_time'] = all_events['deadline_time'].dt.tz_localize(None)
player_future_fixture['kickoff_time'] = player_future_fixture['kickoff_time'].dt.tz_localize(None)

player_gameweek_history['kickoff_time'] = pd.to_datetime(player_gameweek_history['kickoff_time'])
player_gameweek_history['kickoff_time'] = player_gameweek_history['kickoff_time'].dt.tz_localize(None)


## Add Columns

In [14]:
def opponent_team(row):
    if row['is_home'] == True:
        val = row['away_team_id']
    elif row['is_home'] == False:
        val = row['home_team_id']
    return val

def team(row):
    if row['is_home'] == False:
        val = row['away_team_id']
    elif row['is_home'] == True:
        val = row['home_team_id']
    return val


player_future_fixture['opponent_team_id'] = player_future_fixture.apply(opponent_team,axis=1)
player_future_fixture['team_id'] = player_future_fixture.apply(team,axis=1)


## Calculate New Columns

In [15]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,first_name,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,second_name,selected_by_percent,status,team_id,...,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,100.0,100.0,80201,0,0,-5,5,0,1,0.0,1.0,0,Bernd,0.0,1,False,NaN,2022-02-11 08:00:15.144286,45,80201.jpg,1.3,Leno,0.8,a,1,...,0,0,0,0,9,0,48,79.0,0.0,0.0,7.9,362,29,595,69,575,60,417,30,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,0.0,0,Rúnar Alex,0.0,2,False,Joined OH Leuven on a season-long loan - Expec...,2021-08-31 22:00:09.069158,40,115918.jpg,0.0,Rúnarsson,0.6,u,1,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,517,53,505,32,466,19,521,53,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,47431,0,0,-2,2,0,3,0.0,0.0,0,Willian,0.0,3,False,Transferred to Corinthians,2021-08-20 09:30:14.065783,63,47431.jpg,0.0,Borges Da Silva,0.1,u,1,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,706,292,706,292,706,292,706,292,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,54694,0,0,-4,4,0,4,0.0,0.0,0,Pierre-Emerick,0.0,4,False,Left club by mutual consent,2022-02-02 08:21:28.428217,96,54694.jpg,3.1,Aubameyang,1.3,u,1,...,0,2,3,0,0,7,131,217.6,132.4,582.0,92.9,224,27,192,30,28,13,81,21,NaN,NaN,NaN,NaN,NaN,NaN
4,100.0,100.0,58822,0,0,-3,3,0,2,0.0,6.4,2,Cédric,2.7,5,False,NaN,2022-01-13 23:00:13.573747,42,58822.jpg,1.7,Soares,0.2,a,1,...,0,0,1,0,0,0,66,75.0,92.8,23.0,19.1,368,142,236,63,361,132,356,124,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
all_players['PP90'] = np.divide(all_players['total_points'] , all_players['total_minutes'])*90 

In [17]:
all_players['PPMM'] = np.divide(all_players['PPG'] , all_players['now_cost']/10)

In [18]:
all_players['VAPM'] = np.divide(all_players['PPG']-2 , all_players['now_cost']/10)

In [19]:
all_players[['web_name' , 'player_type_id', 'total_points' , 'total_minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['total_minutes']>100].sort_values(by='VAPM' , ascending = False).head()

,web_name,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
199,Chilwell,2,54,540,9.000000,9.0,1.607143,1.250000
489,Trippier,2,26,317,7.381703,6.5,1.300000,0.900000
399,Stones,2,45,591,6.852792,5.6,1.076923,0.692308
414,Torres,3,26,303,7.722772,6.5,0.970149,0.671642
208,Sarr,2,21,343,5.510204,5.2,1.061224,0.653061


## Save cleaned data

In [20]:
all_players.to_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', index = False, header=True)
all_teams.to_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv', index = False, header = True)
all_events.to_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', index = False, header = True)
player_types.to_csv(r'./data/' + season + '/Cleaned_data/player_types.csv', index = False, header = True)
game_phases.to_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv', index = False, header = True)

player_season_history.to_csv (r'./data/' + season + '/Cleaned_data/player_season_history.csv', index = False, header=True)
player_gameweek_history.to_csv (r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv', index = False, header=True)
player_future_fixture.to_csv (r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', index = False, header=True)